In [38]:
import os

os.chdir("/work")

# %% 準備
import pandas as pd
from utils.logger import app_logger
from utils.config_loader import get_template_config
from logic.manage.utils.csv_loader import load_all_filtered_dataframes
from logic.manage.utils.load_template import load_master_and_template
from IPython.display import display
import re
from logic.manage.factory_report import process

In [39]:
from logic.manage.processors.factory_report_yuuka import process_yuuka

In [40]:
# 表示ラベルマップ（処理対象名として使う）
csv_label_map = {"yard": "ヤード一覧", "shipping": "出荷一覧", "receive": "受入一覧"}

debug_shipping = "/work/data/output/debug_shipping.parquet"
debug_yard = "/work/data/output/debug_yard.parquet"

dfs = {
    "shipping": pd.read_parquet(debug_shipping),
    "yard": pd.read_parquet(debug_yard),
}  # テスト用CSV
# dfs
df_shipping = dfs["shipping"]
df_yard = dfs["yard"]

## マスターCSV（有価）の取得

In [41]:
def process_yuuka(df_yard, df_shipping) -> pd.DataFrame:
    logger = app_logger()

    # マスターCSVの読込
    master_path = get_template_config()["factory_report"]["master_csv_path"]["yuuka"]
    master_csv = load_master_and_template(master_path)

    # 各処理を実行
    # updated_master_csv = apply_shipping(master_csv, df_shipping)

    # display(master_csv)
    return master_csv


master_csv1 = process_yuuka(df_yard, df_shipping)

## 有価の処理

In [42]:
import pandas as pd


def merge_safely_with_keys(
    master_df: pd.DataFrame,
    data_df: pd.DataFrame,
    key_cols: list[str],
    how: str = "left",
) -> pd.DataFrame:
    """
    指定した複数のキー列を使って、安全にマージする関数。
    マスター側のキー列に欠損値（NaN）がある行はマージ対象から除外される。

    Parameters
    ----------
    master_df : pd.DataFrame
        マージ元（テンプレート）
    data_df : pd.DataFrame
        マージ対象のデータ
    key_cols : list[str]
        結合に使用するキー列（1〜3列想定）
    how : str
        マージ方法（デフォルト: "left"）

    Returns
    -------
    pd.DataFrame
        マージ済みのDataFrame（未マージ行も含まれる）
    """

    # ① キーに空欄がある行を除外してマージ
    master_valid = master_df.dropna(subset=key_cols)
    data_valid = data_df.dropna(subset=key_cols)

    merged = pd.merge(master_valid, data_valid, on=key_cols, how=how)

    # ② キーが不完全（NaN含む）な行を保持して復元
    master_skipped = master_df[master_df[key_cols].isna().any(axis=1)]

    # ③ マージしたものと未マージのものを結合して返す
    final_df = pd.concat([merged, master_skipped], ignore_index=True)

    return final_df

In [43]:
def update_column_if_present(df, source_col: str, target_col: str) -> pd.DataFrame:
    mask = df[source_col].notna()
    df.loc[mask, target_col] = df.loc[mask, source_col]
    return df

In [44]:
def apply_summary_by_sheetname(
    master_csv: pd.DataFrame,
    data_df: pd.DataFrame,
    sheet_name: str,
    key_cols: list[str],
    source_col: str = "正味重量",
    target_col: str = "値",
) -> pd.DataFrame:
    """
    マスターCSVの特定シート部分に対して、外部データをgroupby集計してマージし、値を書き込む汎用関数。

    Parameters
    ----------
    master_csv : pd.DataFrame
        全体のテンプレートCSV（複数シートを含む）
    data_df : pd.DataFrame
        処理対象のデータ（例：df_shipping）
    sheet_name : str
        処理対象とする "CSVシート名"（例："出荷"）
    key_cols : list[str]
        groupbyキー ＝ マージキー（例：["品名"], ["業者名", "品名"]）
    source_col : str
        集計対象の列（例："正味重量"）
    target_col : str
        書き込み先の列（例："値"）

    Returns
    -------
    pd.DataFrame
        処理済みのマスターCSV（"CSVシート名"以外も含む）
    """
    logger = app_logger()
    logger.info(
        f"▶️ 処理対象シート: {sheet_name}, キー: {key_cols}, 集計列: {source_col}"
    )

    # ① 該当シート部分を取り出す
    target_df = master_csv[master_csv["CSVシート名"] == sheet_name].copy()

    # ② groupbyで合計
    agg_df = data_df.groupby(key_cols, as_index=False)[[source_col]].sum()

    # ③ 安全にマージ
    merged_df = merge_safely_with_keys(
        master_df=target_df, data_df=agg_df, key_cols=key_cols
    )

    # ④ 値を書き込み（NaN以外）
    merged_df = update_column_if_present(merged_df, source_col, target_col)

    # ⑤ 不要列を削除
    merged_df.drop(columns=[source_col], inplace=True)

    # ⑥ 元に戻す：シート以外はそのまま
    master_others = master_csv[master_csv["CSVシート名"] != sheet_name]
    final_df = pd.concat([master_others, merged_df], ignore_index=True)

    return final_df

In [45]:
df_map = {
    "ヤード": df_yard,
    "出荷": df_shipping
}

sheet_key_pairs = [
    ("ヤード", ["品名"]),
    ("出荷", ["品名"]),
    ("出荷", ["業者名", "品名"]),
    ("出荷", ["現場名", "運搬業者名", "品名"]),
]

master_csv_updated = master_csv1.copy()

for sheet_name, key_cols in sheet_key_pairs:
    data_df = df_map[sheet_name]

    master_csv_updated = apply_summary_by_sheetname(
        master_csv=master_csv_updated,
        data_df=data_df,
        sheet_name=sheet_name,
        key_cols=key_cols
    )
master_csv_updated


2025-04-24 14:21:33,804 [INFO] (3211554866.py:33) [fb6959af9847/root] ▶️ 処理対象シート: ヤード, キー: ['品名'], 集計列: 正味重量
2025-04-24 14:21:33,818 [INFO] (3211554866.py:33) [fb6959af9847/root] ▶️ 処理対象シート: 出荷, キー: ['品名'], 集計列: 正味重量
2025-04-24 14:21:33,831 [INFO] (3211554866.py:33) [fb6959af9847/root] ▶️ 処理対象シート: 出荷, キー: ['業者名', '品名'], 集計列: 正味重量
2025-04-24 14:21:33,848 [INFO] (3211554866.py:33) [fb6959af9847/root] ▶️ 処理対象シート: 出荷, キー: ['現場名', '運搬業者名', '品名'], 集計列: 正味重量


,有価名,CSVシート名,業者名,現場名,運搬業者名,品名,セル,値
0,合計,有価,NaN,NaN,NaN,NaN,O21,NaN
1,スクラップ,ヤード,NaN,NaN,NaN,GAH鋼･鉄筋等,C20,NaN
2,スクラップ,ヤード,NaN,NaN,NaN,GC軽鉄･ｽﾁｰﾙ類,C20,NaN
3,スクラップ,ヤード,NaN,NaN,NaN,GC軽鉄・ｽﾁｰﾙ類,C20,NaN
4,スクラップ,ヤード,NaN,NaN,NaN,GD,C20,110
5,アルミ類,ヤード,NaN,NaN,NaN,ｱﾙﾐ類,L20,NaN
6,ステンレス,ヤード,NaN,NaN,NaN,ｽﾃﾝﾚｽ,C22,NaN
7,トランス,ヤード,NaN,NaN,NaN,ﾄﾗﾝｽ,F22,NaN
8,モーター,ヤード,NaN,NaN,NaN,ﾓｰﾀｰ,I22,NaN
9,ラジエター,ヤード,NaN,NaN,NaN,ﾗｼﾞｴﾀｰ,L22,NaN


In [47]:
master_csv_updated["値"] = pd.to_numeric(master_csv_updated["値"], errors="coerce")
grouped_sum = master_csv_updated.groupby("有価名")["値"].sum()
grouped_sum

有価名
Cﾌﾟﾚｽ             0.0
GB(ｷﾞﾛA)      16000.0
アルミ缶             60.0
アルミ類            450.0
スクラップ           670.0
スチール缶             0.0
ステンレス             0.0
トランス              0.0
ペットボトル          190.0
モーター              0.0
ラジエター             0.0
千地             1460.0
合計                0.0
塩ﾋﾞ管(ｸﾞﾚｰ)        0.0
室外機               0.0
紙、ダンボール         250.0
給湯器               0.0
配線             2270.0
銅                 0.0
雑線                0.0
Name: 値, dtype: float64